In [1]:
import pandas as pd
structure = pd.read_csv("course-217-structure.csv")
data = pd.read_csv("course-217-events.csv")

In [2]:
import numpy as np
import numba

In [3]:
structure.head()

,course_id,module_id,module_position,lesson_id,lesson_position,step_id,step_position,step_type,step_cost
0,217,614,2,13228,2,38842,1,text,0
1,217,614,2,13228,2,39715,6,code,1
2,217,614,2,13228,2,39716,7,code,1
3,217,614,2,13228,2,39717,8,code,1
4,217,614,2,13228,2,39721,9,free-answer,0


In [4]:
pos_df = structure.ix[:, [-4, 2, 4, 6]].sort_values("module_position")

In [5]:
from tqdm import tqdm, tqdm_notebook

In [6]:
pos_array = []

for i in range(1, 11):
    for j in range(1, 10):
        for k in range(1, 14):
            pos_array.append([i, j, k])
            
pos_dict = {}
for i, pos in enumerate(pos_array):
    pos_dict[i] = pos

In [7]:
def get_next_step_id(step_id):
    cur_pos = list(pos_df[pos_df.step_id == step_id].ix[:, [1,2,3]].values[0])
#     print(cur_pos)
    
    for key in pos_dict:
        if pos_dict[key] == cur_pos:
            i = key
            break
    
    for j in range(i + 1, 13*9*10+1):
        pos = pos_dict[j]
        if not pos_df.query("(module_position == {0}) & (lesson_position == {1}) & (step_position == {2})".format(*pos)).empty:
            return pos_df.query("(module_position == {0}) & (lesson_position == {1}) & (step_position == {2})".format(*pos)).step_id

In [8]:
def f(x):
    res = []
    try:
        for step_id in np.unique(x.step_id):
            first = second = third = False

            next_step_id = get_next_step_id(step_id).values[0]
            if next_step_id:
                for cur_step_id in x.step_id:
                    if first and second and third:
                        res.append(step_id)
                        break

                    if cur_step_id == step_id and not first:
                        first = True
                    if cur_step_id == next_step_id and not second:
                        second = True
                    if cur_step_id == step_id and not third:
                        third = True
    except:
        pass
    return res

tqdm.pandas()
returnable_users = data.sort_values("time").groupby("user_id").progress_apply(f)

100%|██████████| 7663/7663 [1:08:33<00:00,  1.86it/s]


In [9]:
returnable_users = [np.unique(x) for x in returnable_users]

In [16]:
returnable_users[32].shape

(41,)

In [10]:
from collections import Counter
c = Counter()

for ar in returnable_users:
    c.update(ar)


In [17]:
c.most_common()

[(38518, 6320),
 (38521, 6099),
 (38519, 6042),
 (38529, 5553),
 (38527, 5487),
 (38522, 5340),
 (38523, 5205),
 (38524, 5121),
 (38854, 5114),
 (38525, 5049),
 (38528, 5007),
 (39236, 4951),
 (64865, 4784),
 (38842, 4779),
 (42384, 4180),
 (42385, 4012),
 (42383, 3906),
 (42386, 3817),
 (39715, 3813),
 (39716, 3629),
 (39717, 3430),
 (39721, 3057),
 (38845, 2824),
 (42391, 2617),
 (40665, 2545),
 (42390, 2514),
 (40679, 2500),
 (38848, 2459),
 (42392, 2426),
 (40681, 2359),
 (42397, 2348),
 (40680, 2309),
 (40684, 2296),
 (42396, 2279),
 (38872, 2183),
 (42399, 2180),
 (42398, 2154),
 (42400, 2123),
 (40683, 2106),
 (44383, 2008),
 (44380, 1933),
 (44381, 1853),
 (44382, 1804),
 (39735, 1800),
 (44807, 1775),
 (44808, 1756),
 (39740, 1720),
 (44806, 1715),
 (41216, 1617),
 (38875, 1614),
 (39741, 1581),
 (44387, 1500),
 (44386, 1443),
 (41097, 1411),
 (44385, 1357),
 (38526, 1345),
 (83925, 1300),
 (41099, 1294),
 (38878, 1284),
 (41184, 1266),
 (38897, 1246),
 (41100, 1239),
 (41098,

In [12]:
coeffs = [(key, c[key] / len(data[data.step_id == key].groupby("user_id"))) for key in c]

In [13]:
res = sorted(coeffs, reverse=True, key=lambda x: x[1])[:10]
res
",".join([str(x[0]) for x in res])

[(47438, 0.9887459807073955),
 (44391, 0.9882943143812709),
 (38528, 0.9881586737714624),
 (39236, 0.9846857597454256),
 (48838, 0.9827089337175793),
 (48842, 0.9826589595375722),
 (47290, 0.9823204419889503),
 (42398, 0.9822161422708618),
 (38525, 0.9813411078717201),
 (38524, 0.980846581114729)]

'47438,44391,38528,39236,48838,48842,47290,42398,38525,38524'

In [31]:
len(data[data.step_id == 44120].groupby("user_id"))

1125